In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k


Bad key savefig.frameon in file /Users/vladimir/opt/anaconda3/envs/DataScience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /Users/vladimir/opt/anaconda3/envs/DataScience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /Users/vladimir/opt/anaconda3/envs/DataScience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  #

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)

    return recs.tolist()

In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации

    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    items = np.array(items_weights.item_id)
    weight = np.array(items_weights.weight)
    recs = np.random.choice(items, size=n, replace=False, p=weight)

    return recs.tolist()

In [7]:
popularity = data.groupby('item_id')['sales_value'].count().reset_index().rename(columns={'sales_value': 'sales_count'})
popularity.sort_values('sales_count', ascending=False, inplace=True)
popularity['sales_count_log'] = np.log(popularity['sales_count'])
popularity['weight'] = popularity['sales_count_log'] / popularity['sales_count_log'].sum()
print(f'Сумма weight = {popularity.weight.sum()}')
popularity.head(2)

Сумма weight = 1.0


,item_id,sales_count,sales_count_log,weight
35054,1082185,27362,10.216910,0.000077
56233,6534178,18364,9.818148,0.000074


In [8]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

result.head(2)

CPU times: user 5.56 s, sys: 13.5 ms, total: 5.57 s
Wall time: 5.61 s


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9838023, 872063, 15926886, 13911483, 1022086]","[10344585, 1415996, 1087755, 883670, 1033249]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1065331, 15597054, 10312182, 923149, 1007178]","[8205577, 1115800, 1091090, 1085141, 15596516]"


In [9]:
from metrics import precision_at_k, recall_at_k

for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.0016:weighted_random_recommendation


#### Комментарий
Видим, что по сравнению с обычной случайной рекомендацией, у взвешенной пропорционально количеству продаж, метрика качества в несколько раз выше.

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

#### 2.1 Попробуем улучшить бейзлайны, считая случаный на топ-5000 товаров

In [10]:
%%time

top = 5000

items = popularity.item_id.unique()[:top]

popularity = popularity[:top]
popularity['weight'] = popularity['sales_count_log'] / popularity['sales_count_log'].sum()
print(f'Сумма weight = {popularity.weight.sum()}')

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

result.head(2)

Сумма weight = 0.9999999999999999
CPU times: user 1.58 s, sys: 6.78 ms, total: 1.58 s
Wall time: 1.59 s


,user_id,actual,random_recommendation,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9838023, 872063, 15926886, 13911483, 1022086]","[10344585, 1415996, 1087755, 883670, 1033249]","[891028, 932298, 1054945, 871433, 995916]","[1081189, 1132280, 1038692, 992529, 835619]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1065331, 15597054, 10312182, 923149, 1007178]","[8205577, 1115800, 1091090, 1085141, 15596516]","[13007355, 8090532, 961747, 996425, 1127470]","[959316, 950756, 926763, 910151, 911622]"


In [11]:
from metrics import precision_at_k, recall_at_k

for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.0016:weighted_random_recommendation
0.0059:random_recommendation_5000
0.0057:weighted_random_recommendation_5000


#### Комментарий

Видим, что те же случайные рекомендации, но рассчитанные на top-5000 товаров существенно улучшают бейзлайн.

#### 2.2 Попробуем улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [12]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [13]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [14]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666

/Users/vladimir/opt/anaconda3/envs/DataScience/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [15]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                  )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

print(f'Размер матрицы: {user_item_matrix.shape}')
print(f'Разреженность матрицы: {user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100}')

Размер матрицы: (2499, 5001)
Разреженность матрицы: 5.33770796861036


In [16]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.08 s, sys: 11.9 ms, total: 1.09 s
Wall time: 1.13 s


In [18]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model.recommend(userid=userid_to_id[x],
                                     user_items=sparse_user_item,   # на вход user-item matrix
                                     N=5,
                                     filter_already_liked_items=False,
                                     filter_items=[itemid_to_id[6666]],
                                     recalculate_user=True)])

CPU times: user 44.8 ms, sys: 1.49 ms, total: 46.2 ms
Wall time: 45.8 ms


In [19]:
result.head(5)

,user_id,actual,random_recommendation,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9838023, 872063, 15926886, 13911483, 1022086]","[10344585, 1415996, 1087755, 883670, 1033249]","[891028, 932298, 1054945, 871433, 995916]","[1081189, 1132280, 1038692, 992529, 835619]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1065331, 15597054, 10312182, 923149, 1007178]","[8205577, 1115800, 1091090, 1085141, 15596516]","[13007355, 8090532, 961747, 996425, 1127470]","[959316, 950756, 926763, 910151, 911622]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1069183, 10203856, 9368046, 9526790, 6552364]","[7409956, 982785, 1056187, 1110119, 989075]","[1007911, 936355, 1012922, 977798, 1001333]","[9527455, 896444, 850157, 9527257, 918684]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[15717072, 15716286, 2861517, 998489, 819828]","[12172002, 878444, 921042, 998567, 9396498]","[857849, 1106428, 9245108, 1131558, 1093413]","[1024051, 7155863, 6533889, 1105917, 5568729]","[1082185, 995242, 1029743, 826249, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5655290, 1260411, 879986, 13775118, 1032688]","[906502, 7167984, 9677102, 9396716, 2483297]","[1037417, 825249, 999250, 868015, 920978]","[835300, 962615, 8205218, 979566, 1083733]","[1082185, 981760, 995242, 1029743, 840361]"


In [20]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.0016:weighted_random_recommendation
0.0059:random_recommendation_5000
0.0057:weighted_random_recommendation_5000
0.2199:own_purchases


/Users/vladimir/DataspellProjects/GB-Recommender-Systems/Lesson_2/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


#### Попробуем улучшить результат

In [21]:
# изменим значения в матрице (оставим quantity "как есть")

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                  )

# user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

print(f'Размер матрицы: {user_item_matrix.shape}')
print(f'Разреженность матрицы: {user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100}')

Размер матрицы: (2499, 5001)
Разреженность матрицы: 18.23156777207984


In [22]:
%%time

# применим в модели K=3

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.08 s, sys: 13.6 ms, total: 1.09 s
Wall time: 1.1 s


In [23]:
%%time

result['own_purchases_update'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model.recommend(userid=userid_to_id[x],
                                     user_items=sparse_user_item,   # на вход user-item matrix
                                     N=5,
                                     filter_already_liked_items=False,
                                     filter_items=[itemid_to_id[6666]],
                                     recalculate_user=True)])

CPU times: user 43.7 ms, sys: 1.46 ms, total: 45.2 ms
Wall time: 44.6 ms


In [24]:
result.head(5)

,user_id,actual,random_recommendation,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000,own_purchases,own_purchases_update
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9838023, 872063, 15926886, 13911483, 1022086]","[10344585, 1415996, 1087755, 883670, 1033249]","[891028, 932298, 1054945, 871433, 995916]","[1081189, 1132280, 1038692, 992529, 835619]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 6534178, 995242, 1029743, 1005186]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1065331, 15597054, 10312182, 923149, 1007178]","[8205577, 1115800, 1091090, 1085141, 15596516]","[13007355, 8090532, 961747, 996425, 1127470]","[959316, 950756, 926763, 910151, 911622]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 6534178, 1106523, 951590, 1133018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1069183, 10203856, 9368046, 9526790, 6552364]","[7409956, 982785, 1056187, 1110119, 989075]","[1007911, 936355, 1012922, 977798, 1001333]","[9527455, 896444, 850157, 9527257, 918684]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 6534178, 1029743, 995242, 5569230]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[15717072, 15716286, 2861517, 998489, 819828]","[12172002, 878444, 921042, 998567, 9396498]","[857849, 1106428, 9245108, 1131558, 1093413]","[1024051, 7155863, 6533889, 1105917, 5568729]","[1082185, 995242, 1029743, 826249, 1127831]","[1082185, 6534178, 1106523, 1029743, 1133018]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5655290, 1260411, 879986, 13775118, 1032688]","[906502, 7167984, 9677102, 9396716, 2483297]","[1037417, 825249, 999250, 868015, 920978]","[835300, 962615, 8205218, 979566, 1083733]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 6534178, 1029743, 1106523, 1133018]"


In [25]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.0016:weighted_random_recommendation
0.0059:random_recommendation_5000
0.0057:weighted_random_recommendation_5000
0.2199:own_purchases
0.3055:own_purchases_update


/Users/vladimir/DataspellProjects/GB-Recommender-Systems/Lesson_2/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


#### Итого

После серии экспериментов лучший результат 0.3055. В матрице используем количество совершенных покупок (не 0/1), K=3.